In [1]:
import matplotlib.pyplot as plt
import simpy
import numpy as np
from sympy import *
import statistics
import random

In [3]:

class Myqueue:
    def __init__(self, env, lam, mu_2):
        self.env = env
        self.server_1 = simpy.Resource(self.env, 10000)
        self.q_2 = simpy.Store(env)
        self.mean_arrival = 1/lam
        self.mean_service = 1/mu_2
        self.n = 0
        self.data = {'Arrival D':[], 'Departure D':[], 'Arrival I':[], 'Departure I':[], 'idle':[0], 'delay':[]}


    def queue_1(self):
            x = np.longfloat(self.env.now)
            self.data['Arrival I'].append(x)
            self.q_2.put('CI')
            with self.server_1.request() as request:
                yield request
                yield self.env.timeout(random.expovariate(0.5))
                #np.random.pareto(a=2)
                #np.longfloat(np.random.uniform(0,10)) + np.longfloat(np.random.uniform(0,10)) + random.expovariate(random.choice((0.1,2,3)))
                #np.longfloat(np.random.uniform(0,10))
                #random.expovariate(random.choice((0.1,2,3)))
            self.q_2.put('CD')
            y = np.longfloat(self.env.now)
            self.data['Arrival D'].append(y)
        
            if len(self.data['Arrival D']) > 10**5:
                if x < self.data['idle'][-1] < y:
                    self.n = self.n + 1
                
    def queue_2(self):
        while True:
            y = yield self.q_2.get()
            if y == 'CI':
                if len(self.q_2.items) == 0:
                    self.q_2.put('1')
                    yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service)))
                    self.data['Departure I'].append(np.longfloat(self.env.now))
                    self.q_2.get()
                else:
                    yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service)))
                    self.data['Departure I'].append(np.longfloat(self.env.now))
            else:
                if len(self.q_2.items) == 0:
                    self.q_2.put('1')
                    yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service)))
                    self.data['Departure D'].append(np.longfloat(self.env.now))
                    self.q_2.get()
                else:
                    yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service)))
                    self.data['Departure D'].append(np.longfloat(self.env.now))

            if len(self.q_2.items) == 0:
                self.data['idle'].append(np.longfloat(self.env.now))


    def arrival(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_arrival)))
            self.env.process(self.queue_1())


In [5]:
lam, mu, n = 0.3, 1, 2*10**6
env = simpy.Environment()
net = Myqueue(env, lam, mu)
env.process(net.arrival())
env.process(net.queue_2())
env.run(n) 

In [6]:
l_1 = len(net.data['Departure D'])
l_2 = len(net.data['Departure I'])

l1 = int(l_1*0.3)   #margin
l2 = int(l_2*0.3)   #margin

W_D = (sum(net.data['Departure D'][l1:-l1+l_1]) - sum(net.data['Arrival D'][l1:-l1+l_1])) / len(net.data['Departure D'][l1:-l1+l_1])
W_I = (sum(net.data['Departure I'][l2:-l2+l_2]) - sum(net.data['Arrival I'][l2:-l2+l_2])) / len(net.data['Departure I'][l2:-l2+l_2])
print(W_D, W_I, W_D-W_I)

3.6741587087940775 3.007769635729185 0.6663890730648925


In [7]:
probability = 1-net.n/len(net.data['Arrival I'][10**5:])

In [8]:
print('Error = ',(probability - (W_D-W_I)*mu)/probability*100, ',     probability =', probability)

Error =  0.11667789727837916 ,     probability = 0.6671675100869865
